# Insurance Analytics - Week 3 EDA Notebook
- A Comprehensive Exploratory Data Analysis 

1. INITIAL SETUP & IMPORTS

In [ ]:
# Insurance Analytics - Week 3 EDA Notebook
# A Comprehensive Exploratory Data Analysis
"""
Insurance Analytics - Week 3 EDA Notebook
Comprehensive Exploratory Data Analysis
"""

# ============================================================================
# 1. INITIAL SETUP & IMPORTS
# ============================================================================
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Add project root to path
current_dir = os.getcwd()
if os.path.basename(current_dir) == 'notebooks':
    project_root = os.path.dirname(current_dir)
    os.chdir(project_root)
else:
    project_root = current_dir

if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"📂 Project Root: {project_root}")
print(f"📁 Working Directory: {os.getcwd()}")

# Core imports
import pandas as pd
import numpy as np
from datetime import datetime
import json
import matplotlib.pyplot as plt
import seaborn as sns

# Set visual styles
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams.update({
    'figure.figsize': (12, 6),
    'font.size': 12,
    'axes.titlesize': 14,
    'axes.labelsize': 12
})

# Import custom modules
try:
    from src.data_preprocessing import DataPreprocessor
    from src.utils import DataUtils
    from src.visualization import DataVisualizer
    # Try to import eda, but handle if it has issues
    try:
        from src.eda import InsuranceEDA
        eda_available = True
    except ImportError as e:
        print(f"⚠️ Could not import InsuranceEDA: {e}")
        print("Will use DataUtils and DataVisualizer directly")
        eda_available = False
    print("✅ Successfully imported custom modules")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Please ensure your src modules are accessible")
    eda_available = False


In [ ]:

# ============================================================================
# 2. DATA LOADING & PREPROCESSING WITH TOTALCLAIMS FIX
# ============================================================================
print("\n" + "="*70)
print("DATA LOADING & PREPROCESSING")
print("="*70)

# Define paths
RAW_DATA_PATH = "D:/Python/Week-3/Raw_Data/MachineLearningRating_v3.txt"
PROCESSED_DATA_PATH = "D:/Python/Week-3/Insurance-Analytics-Week-3-/data/processed/processed_MachineLearningRating_v3.csv"
CLAIMS_DATA_PATH = "D:/Python/Week-3/Insurance-Analytics-Week-3-/data/processed/claims_positive_MachineLearningRating_v3.csv"

print("\n🔍 CHOOSE DATASET TO ANALYZE:")
print("1. Full dataset (all policies)")
print("2. Claims dataset (only policies with TotalClaims > 0)")
choice = input("\nEnter choice (1 or 2): ").strip()

if choice == "2":
    USE_CLAIMS_ONLY = True
    PROCESSED_DATA_PATH = CLAIMS_DATA_PATH
    print("\n📊 Will analyze CLAIMS DATA ONLY (TotalClaims > 0)")
else:
    USE_CLAIMS_ONLY = False
    print("\n📊 Will analyze FULL DATASET (all policies)")

# Check if processed data exists
if os.path.exists(PROCESSED_DATA_PATH):
    print(f"\n📁 Loading processed data from: {PROCESSED_DATA_PATH}")
    try:
        df = pd.read_csv(PROCESSED_DATA_PATH, sep='|')
        print(f"✅ Loaded {len(df):,} rows, {len(df.columns)} columns")
        if USE_CLAIMS_ONLY:
            print("📈 Analyzing ONLY policies with claims (TotalClaims > 0)")
        else:
            print("📈 Analyzing ALL policies")
    except Exception as e:
        print(f"❌ Error loading processed data: {e}")
        print("🔧 Falling back to preprocessing...")
        df = None
else:
    print("🔍 Processed data not found. Starting preprocessing...")
    df = None

# Preprocess if needed
if df is None:
    print("\n🔄 Initializing DataPreprocessor...")
    
    if USE_CLAIMS_ONLY:
        print("🔍 Will filter for TotalClaims > 0 during processing")
        claims_filter = "positive"
        out_path = CLAIMS_DATA_PATH
    else:
        claims_filter = None
        out_path = PROCESSED_DATA_PATH
    
    dp = DataPreprocessor(
        raw_path=RAW_DATA_PATH,
        out_path=out_path,
        chunksize=100_000,
        delimiter="|",
        log_transform=True,
        claims_filter=claims_filter
    )
    
    print("🔧 Processing data with enhanced features...")
    df = dp.process(
        save_format="csv",
        create_features=True,
        run_quality_checks=True
    )
    print("✅ Preprocessing completed successfully!")


In [ ]:

# ============================================================================
# 3. FIX TOTALCLAIMS CONVERSION ISSUE
# ============================================================================
print("\n" + "="*70)
print("FIXING TOTALCLAIMS CONVERSION ISSUE")
print("="*70)

# Check current TotalClaims values
if 'TotalClaims' in df.columns:
    print(f"\n🔍 CURRENT TOTALCLAIMS STATUS:")
    print(f"• Data type: {df['TotalClaims'].dtype}")
    print(f"• Total policies: {len(df):,}")
    print(f"• Policies with claims > 0: {(df['TotalClaims'] > 0).sum():,}")
    print(f"• Total claims amount: ${df['TotalClaims'].sum():,.2f}")
    
    # If all claims are zero but we expect some, fix the conversion
    if (df['TotalClaims'] > 0).sum() == 0:
        print("\n⚠️ WARNING: All TotalClaims are zero!")
        print("🔧 Fixing TotalClaims conversion from raw data...")
        
        # Function to properly convert TotalClaims values
        def fix_totalclaims_conversion():
            """Reload TotalClaims from raw data with proper conversion"""
            claims_list = []
            chunksize = 100000
            rows_processed = 0
            
            print(f"  Loading TotalClaims from raw data: {RAW_DATA_PATH}")
            
            # Read raw data in chunks
            for chunk in pd.read_csv(RAW_DATA_PATH, sep='|', 
                                    chunksize=chunksize,
                                    usecols=['TotalClaims'],
                                    dtype=str):
                
                # Convert each value properly
                for val in chunk['TotalClaims']:
                    if pd.isna(val) or val == '':
                        claims_list.append(0.0)
                    elif val == '.000000000000':
                        claims_list.append(0.0)
                    elif val.startswith('.') and val[1:].replace('0', '').replace('.', '') == '':
                        # Handle cases like '.000000' or '.0'
                        claims_list.append(0.0)
                    else:
                        try:
                            # Remove any non-numeric characters except decimal point
                            clean_val = ''.join(c for c in val if c.isdigit() or c == '.' or c == '-')
                            if clean_val and clean_val != '.':
                                claims_list.append(float(clean_val))
                            else:
                                claims_list.append(0.0)
                        except:
                            claims_list.append(0.0)
                
                rows_processed += len(chunk)
                if rows_processed % 500000 == 0:
                    print(f"    Processed {rows_processed:,} rows...")
            
            return claims_list
        
        # Get the properly converted claims
        print("  Converting TotalClaims values...")
        fixed_claims = fix_totalclaims_conversion()
        
        # Ensure we have the right number of rows
        if len(fixed_claims) != len(df):
            print(f"  ⚠️ Warning: Length mismatch. Raw has {len(fixed_claims)} rows, df has {len(df)} rows")
            # Truncate or pad as needed
            if len(fixed_claims) > len(df):
                fixed_claims = fixed_claims[:len(df)]
            else:
                fixed_claims.extend([0.0] * (len(df) - len(fixed_claims)))
        
        # Replace the column
        df['TotalClaims_Original'] = df['TotalClaims']  # Keep original for comparison
        df['TotalClaims'] = fixed_claims
        
        print(f"\n✅ TOTALCLAIMS AFTER FIX:")
        print(f"• Data type: {df['TotalClaims'].dtype}")
        print(f"• Policies with claims > 0: {(df['TotalClaims'] > 0).sum():,} ({(df['TotalClaims'] > 0).sum()/len(df)*100:.2f}%)")
        print(f"• Total claims amount: ${df['TotalClaims'].sum():,.2f}")
        
        if (df['TotalClaims'] > 0).sum() > 0:
            claims_data = df[df['TotalClaims'] > 0]['TotalClaims']
            print(f"• Average claim amount: ${claims_data.mean():,.2f}")
            print(f"• Median claim amount: ${claims_data.median():,.2f}")
            print(f"• Max claim amount: ${claims_data.max():,.2f}")
            print(f"• Min claim amount (>0): ${claims_data.min():,.2f}")
    else:
        print("\n✅ TotalClaims already has positive values!")
else:
    print("⚠️ TotalClaims column not found in dataset")

# Quick check of dataset
print(f"\n📊 Dataset shape: {df.shape}")
print(f"🧠 Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")


In [ ]:

# ============================================================================
# 4. UPDATE DERIVED COLUMNS AFTER FIX
# ============================================================================
print("\n" + "="*70)
print("UPDATING DERIVED COLUMNS")
print("="*70)

# Recalculate LossRatio if needed
if 'TotalClaims' in df.columns and 'TotalPremium' in df.columns:
    print("\n🔄 Recalculating LossRatio...")
    
    # Calculate LossRatio
    df['LossRatio'] = df['TotalClaims'] / df['TotalPremium'].replace(0, np.nan)
    df['LossRatio'] = df['LossRatio'].clip(lower=0, upper=10)  # Cap at 1000%
    
    # Calculate claim indicators
    df['HasClaim'] = (df['TotalClaims'] > 0).astype('int8')
    
    print(f"✅ Derived columns updated:")
    print(f"• Average Loss Ratio: {df['LossRatio'].mean():.2%}")
    print(f"• Claim Frequency: {df['HasClaim'].mean():.2%}")
    
    if (df['TotalClaims'] > 0).sum() > 0:
        print(f"• Average Loss Ratio (claims only): {df[df['TotalClaims'] > 0]['LossRatio'].mean():.2%}")


In [ ]:

# ============================================================================
# 4. EDA IMPLEMENTATION
# ============================================================================
print("\n" + "="*70)
print("RUBRIC 1: EDA IMPLEMENTATION")
print("="*70)

# Initialize utility objects
utils = DataUtils()
viz = DataVisualizer()

# Initialize EDA if available
if eda_available:
    eda = InsuranceEDA(df)
    print("✅ Using InsuranceEDA class for analysis")
else:
    print("⚠️ Using direct methods for analysis (InsuranceEDA not available)")
    eda = None


# ----------------------------------------------------------------------------
# 1.1 DATA SUMMARIZATION (Rubric 1.1)
# ----------------------------------------------------------------------------
print("\n📋 1.1 DATA SUMMARIZATION")
print("-" * 40)

# Data structure summary using DataUtils
print("📊 DATA STRUCTURE SUMMARY")
print("-" * 30)
print(f"• Rows: {df.shape[0]:,}")
print(f"• Columns: {df.shape[1]}")
print(f"• Memory: {utils.memory_usage(df):.2f} MB")

if USE_CLAIMS_ONLY:
    print(f"• Dataset Type: Claims Data Only (TotalClaims > 0)")

print("\n📊 DATA TYPES:")
dtype_counts = df.dtypes.value_counts()
for dtype, count in dtype_counts.items():
    print(f"  • {dtype}: {count} columns")

# Enhanced descriptive statistics
print("\n📈 ENHANCED DESCRIPTIVE STATISTICS")
print("-" * 40)

# Key numerical features for insurance analysis
key_numerical = ['TotalPremium', 'TotalClaims', 'SumInsured', 
                 'CalculatedPremiumPerTerm', 'LossRatio', 'VehicleAge',
                 'PremiumRate', 'HasClaim']

existing_numerical = [col for col in key_numerical if col in df.columns]
if existing_numerical:
    desc_stats = df[existing_numerical].describe().T.round(2)
    
    # Add additional statistics
    desc_stats['skewness'] = df[existing_numerical].skew().round(3)
    desc_stats['kurtosis'] = df[existing_numerical].kurtosis().round(3)
    desc_stats['zeros'] = (df[existing_numerical] == 0).sum()
    desc_stats['zeros_pct'] = ((df[existing_numerical] == 0).sum() / len(df) * 100).round(2)
    
    print("\nDescriptive Statistics for Key Insurance Features:")
    print(desc_stats)
    
    # Save to file for documentation
    os.makedirs('reports', exist_ok=True)
    desc_stats.to_csv('reports/descriptive_statistics.csv')
    print("💾 Saved descriptive statistics to 'reports/descriptive_statistics.csv'")

# ----------------------------------------------------------------------------
# 1.2 DATA QUALITY ASSESSMENT (Rubric 1.2)
# ----------------------------------------------------------------------------
print("\n🔍 1.2 DATA QUALITY ASSESSMENT")
print("-" * 40)

# Missing value analysis
print("❓ MISSING VALUES ANALYSIS")
print("-" * 30)
missing = df.isna().sum()
missing_pct = missing / len(df) * 100
missing_df = pd.DataFrame({"Missing": missing, "Percent": missing_pct})
missing_df = missing_df[missing_df["Missing"] > 0]

if len(missing_df) > 0:
    print("Columns with missing values:")
    print(missing_df.sort_values("Percent", ascending=False).head(10))
else:
    print("✅ No missing values found!")

# Enhanced quality report
print("\n📋 ENHANCED DATA QUALITY REPORT")
print("-" * 40)

# Check for duplicates
duplicate_count = df.duplicated().sum()
print(f"• Duplicate rows: {duplicate_count:,} ({duplicate_count/len(df)*100:.2f}%)")

# Check for invalid values in key columns
print("\n🔎 INVALID VALUES CHECK:")
key_columns = {
    'TotalPremium': (df['TotalPremium'] < 0).sum() if 'TotalPremium' in df.columns else 0,
    'TotalClaims': (df['TotalClaims'] < 0).sum() if 'TotalClaims' in df.columns else 0,
    'LossRatio': ((df['LossRatio'] < 0) | (df['LossRatio'] > 10)).sum() if 'LossRatio' in df.columns else 0
}

for col, count in key_columns.items():
    if count > 0:
        print(f"  [WARNING] {col}: {count:,} invalid values")
    else:
        print(f"  [OK] {col}: No invalid values")

# Special check for claims data
if USE_CLAIMS_ONLY and 'TotalClaims' in df.columns:
    print("\n🔍 CLAIMS DATA SPECIFIC CHECK:")
    zero_claims = (df['TotalClaims'] == 0).sum()
    if zero_claims > 0:
        print(f"  [WARNING] Found {zero_claims} rows with TotalClaims = 0 in claims-only dataset")
    else:
        print(f"  [OK] All rows have TotalClaims > 0")

# ----------------------------------------------------------------------------
# 1.3 UNIVARIATE ANALYSIS (Rubric 1.3)
# ----------------------------------------------------------------------------
print("\n📊 1.3 UNIVARIATE ANALYSIS")
print("-" * 40)

# Get numerical and categorical columns
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"• Numerical columns to analyze: {len(numerical_cols)}")
print(f"• Categorical columns to analyze: {len(categorical_cols)}")

# Select top features for visualization
top_numerical = numerical_cols[:6] if len(numerical_cols) > 6 else numerical_cols
print(f"\n📈 HISTOGRAMS FOR: {', '.join(top_numerical)}")

# Create histograms in a grid
if len(top_numerical) > 0:
    n_cols = min(3, len(top_numerical))
    n_rows = (len(top_numerical) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 4*n_rows))
    axes = axes.flatten() if n_rows > 1 or n_cols > 1 else [axes]
    
    for idx, col in enumerate(top_numerical):
        ax = axes[idx]
        data = df[col].dropna()
        
        if len(data) > 0:
            ax.hist(data, bins=50, alpha=0.7, color='steelblue', edgecolor='black')
            ax.set_title(f'Distribution of {col}', fontsize=11)
            ax.set_xlabel(col, fontsize=9)
            ax.set_ylabel('Frequency', fontsize=9)
            ax.grid(True, alpha=0.3)
            
            # Add statistics
            stats_text = f"Mean: {data.mean():.2f}\nStd: {data.std():.2f}"
            if USE_CLAIMS_ONLY and col == 'TotalClaims':
                stats_text += f"\nClaims Only"
            ax.text(0.95, 0.95, stats_text, transform=ax.transAxes,
                   fontsize=8, verticalalignment='top', horizontalalignment='right',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # Hide empty subplots
    for idx in range(len(top_numerical), len(axes)):
        axes[idx].set_visible(False)
    
    title = 'Univariate Analysis: Numerical Feature Distributions'
    if USE_CLAIMS_ONLY:
        title += ' (Claims Data Only)'
    plt.suptitle(title, fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()

# Bar charts for categorical features
top_categorical = categorical_cols[:4] if len(categorical_cols) > 4 else categorical_cols
print(f"\n📊 BAR CHARTS FOR: {', '.join(top_categorical)}")

for col in top_categorical:
    plt.figure(figsize=(12, 6))
    value_counts = df[col].value_counts().head(15)  # Top 15 categories
    value_counts.plot(kind='bar', color='lightcoral', edgecolor='black')
    title = f'Distribution of {col} (Top 15)'
    if USE_CLAIMS_ONLY:
        title += ' (Claims Data)'
    plt.title(title, fontsize=14)
    plt.xlabel(col, fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.grid(True, alpha=0.3, axis='y')
    
    # Add value labels
    for i, (idx, val) in enumerate(value_counts.items()):
        plt.text(i, val + max(value_counts)*0.01, f'{val:,}', 
                ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.show()

# ----------------------------------------------------------------------------
# 1.4 BIVARIATE/MULTIVARIATE ANALYSIS (Rubric 1.4)
# ----------------------------------------------------------------------------
print("\n🔗 1.4 BIVARIATE/MULTIVARIATE ANALYSIS")
print("-" * 40)

# 1.4.1 Correlation Analysis (REQUIRED)
print("\n📊 CORRELATION MATRIX ANALYSIS")
print("-" * 40)

# Select key numerical features for correlation
correlation_features = ['TotalPremium', 'TotalClaims', 'SumInsured', 
                       'CalculatedPremiumPerTerm', 'VehicleAge', 'LossRatio',
                       'PremiumRate']

available_features = [col for col in correlation_features if col in df.columns]

if len(available_features) >= 3:
    print(f"Calculating correlations for: {', '.join(available_features)}")
    
    # Calculate correlation matrix
    corr_matrix = df[available_features].corr()
    
    # Plot heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
    title = 'Correlation Matrix of Key Insurance Features'
    if USE_CLAIMS_ONLY:
        title += ' (Claims Data Only)'
    plt.title(title, fontsize=14, pad=20)
    plt.tight_layout()
    plt.show()
    
    print("\n💪 STRONGEST CORRELATIONS (|r| > 0.5)")
    print("-" * 30)
    
    # Find strong correlations
    strong_correlations = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            corr_value = corr_matrix.iloc[i, j]
            if abs(corr_value) > 0.5:
                strong_correlations.append((
                    corr_matrix.columns[i],
                    corr_matrix.columns[j],
                    corr_value
                ))
    
    if strong_correlations:
        for col1, col2, corr_val in sorted(strong_correlations, key=lambda x: abs(x[2]), reverse=True):
            direction = "positive" if corr_val > 0 else "negative"
            print(f"• {col1} ↔ {col2}: {corr_val:.3f} ({direction})")
    else:
        print("No strong correlations (|r| > 0.5) found among selected features")
    
    # Save correlation matrix
    corr_matrix.to_csv('reports/correlation_matrix.csv')
    print("💾 Saved correlation matrix to 'reports/correlation_matrix.csv'")
else:
    print(f"⚠️ Need at least 3 numerical features for correlation matrix. Found: {len(available_features)}")

# 1.4.2 Enhanced Scatter Plots with Fixes
print("\n📍 ENHANCED SCATTER PLOTS")
print("-" * 40)

# Create scatter plots for important relationships
scatter_pairs = [
    ('TotalPremium', 'TotalClaims'),
    ('SumInsured', 'CalculatedPremiumPerTerm'),
    ('VehicleAge', 'LossRatio')
]

for x_col, y_col in scatter_pairs:
    if x_col in df.columns and y_col in df.columns:
        print(f"\nScatter plot: {x_col} vs {y_col}")
        print("-" * 30)
        
        # Filter out NaN values
        valid_data = df[[x_col, y_col]].dropna()
        
        if len(valid_data) > 10:  # Need enough data points
            print(f"• Data points: {len(valid_data):,}")
            print(f"• {x_col} range: [{valid_data[x_col].min():.2f}, {valid_data[x_col].max():.2f}]")
            print(f"• {y_col} range: [{valid_data[y_col].min():.2f}, {valid_data[y_col].max():.2f}]")
            
            # Check if data needs scaling
            if valid_data[y_col].max() - valid_data[y_col].min() < 0.1:
                print(f"⚠️ Warning: {y_col} has very small range. Checking scaling...")
                
                # Try to identify if values are in thousands/millions
                mean_val = valid_data[y_col].mean()
                if abs(mean_val) < 1:
                    print(f"  • Values appear to be in decimal format")
                    print(f"  • Consider if values need to be multiplied (e.g., by 1000)")
            
            # Calculate correlation
            correlation = valid_data[x_col].corr(valid_data[y_col])
            print(f"• Correlation: {correlation:.4f}")
            
            plt.figure(figsize=(12, 8))
            
            # Create scatter plot with better visualization
            if len(valid_data) > 1000:
                # Use hexbin for large datasets
                plt.hexbin(valid_data[x_col], valid_data[y_col], 
                          gridsize=50, cmap='Blues', mincnt=1)
                plt.colorbar(label='Count')
            else:
                # Regular scatter for smaller datasets
                plt.scatter(valid_data[x_col], valid_data[y_col], 
                          alpha=0.5, s=20, color='steelblue', edgecolor='black')
            
            # Add trend line if we have valid correlation
            if abs(correlation) > 0.1:  # Only add if there's some correlation
                try:
                    z = np.polyfit(valid_data[x_col], valid_data[y_col], 1)
                    p = np.poly1d(z)
                    
                    # Generate x values for trend line
                    x_vals = np.linspace(valid_data[x_col].min(), 
                                        valid_data[x_col].max(), 100)
                    plt.plot(x_vals, p(x_vals), "r--", alpha=0.8, 
                            label=f'Trend: y = {z[0]:.6f}x + {z[1]:.4f}')
                    plt.legend()
                except:
                    print("  ⚠️ Could not calculate trend line")
            
            title = f'{x_col} vs {y_col}'
            if USE_CLAIMS_ONLY:
                title += ' (Claims Data)'
            plt.title(title, fontsize=14)
            plt.xlabel(x_col, fontsize=12)
            plt.ylabel(y_col, fontsize=12)
            plt.grid(True, alpha=0.3)
            
            # Add statistics box
            stats_text = (f'Correlation: {correlation:.4f}\n'
                         f'Data points: {len(valid_data):,}\n'
                         f'Mean {x_col}: {valid_data[x_col].mean():.2f}\n'
                         f'Mean {y_col}: {valid_data[y_col].mean():.2f}')
            
            plt.text(0.05, 0.95, stats_text, transform=plt.gca().transAxes, 
                    fontsize=10, verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
            
            plt.tight_layout()
            plt.show()
            
            # Also create a log-scale version if data spans large ranges
            if (valid_data[x_col].max() / valid_data[x_col].min() > 100 or 
                valid_data[y_col].max() / valid_data[y_col].min() > 100):
                
                print(f"📈 Creating log-scale version for better visualization")
                
                plt.figure(figsize=(12, 8))
                plt.scatter(valid_data[x_col], valid_data[y_col], 
                          alpha=0.5, s=20, color='steelblue', edgecolor='black')
                
                plt.xscale('log')
                plt.yscale('log')
                plt.title(f'{x_col} vs {y_col} (Log Scale)', fontsize=14)
                plt.xlabel(f'{x_col} (log scale)', fontsize=12)
                plt.ylabel(f'{y_col} (log scale)', fontsize=12)
                plt.grid(True, alpha=0.3)
                plt.tight_layout()
                plt.show()
        else:
            print(f"⚠️ Insufficient data for scatter plot: only {len(valid_data)} valid points")
    else:
        print(f"⚠️ Cannot plot {x_col} vs {y_col}: columns not found")

# ----------------------------------------------------------------------------
# 1.5 OUTLIER DETECTION (Rubric 1.5)
# ----------------------------------------------------------------------------
print("\n🚨 1.5 OUTLIER DETECTION")
print("-" * 40)

print("\n📦 ENHANCED OUTLIER DETECTION WITH BOX PLOTS")
print("-" * 40)

# Focus on key insurance columns
key_outlier_cols = ['TotalPremium', 'TotalClaims', 'SumInsured', 
                   'CalculatedPremiumPerTerm', 'LossRatio', 'VehicleAge']

available_outlier_cols = [col for col in key_outlier_cols if col in df.columns]

if available_outlier_cols:
    print(f"Generating box plots for: {', '.join(available_outlier_cols)}")
    
    # Create subplots
    n_cols = len(available_outlier_cols)
    n_rows = (n_cols + 2) // 3  # Max 3 per row
    
    fig, axes = plt.subplots(n_rows, 3, figsize=(15, 4*n_rows))
    axes = axes.flatten() if n_rows > 1 else [axes]
    
    outlier_report = {}
    
    for idx, col in enumerate(available_outlier_cols):
        ax = axes[idx]
        data = df[col].dropna()
        
        if len(data) > 0:
            # Calculate IQR and outliers
            Q1 = data.quantile(0.25)
            Q3 = data.quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            outliers = ((data < lower_bound) | (data > upper_bound)).sum()
            outlier_pct = (outliers / len(data) * 100) if len(data) > 0 else 0
            
            outlier_report[col] = {
                'outliers': int(outliers),
                'outlier_pct': round(outlier_pct, 2),
                'Q1': round(Q1, 2),
                'Q3': round(Q3, 2),
                'IQR': round(IQR, 2)
            }
            
            # Create box plot
            box = ax.boxplot(data, vert=True, patch_artist=True,
                            boxprops=dict(facecolor='lightblue', color='darkblue'),
                            medianprops=dict(color='red', linewidth=2),
                            whiskerprops=dict(color='darkblue'),
                            capprops=dict(color='darkblue'),
                            flierprops=dict(marker='o', color='red', alpha=0.5, markersize=4))
            
            title = f'{col}\nOutliers: {outliers} ({outlier_pct:.1f}%)'
            ax.set_title(title, fontsize=11)
            ax.set_ylabel('Value', fontsize=9)
            ax.grid(True, alpha=0.3, axis='y')
        else:
            ax.text(0.5, 0.5, 'No Data', ha='center', va='center', transform=ax.transAxes)
            ax.set_title(col, fontsize=11)
    
    # Hide empty subplots
    for idx in range(len(available_outlier_cols), len(axes)):
        axes[idx].set_visible(False)
    
    title = 'Outlier Detection with Box Plots (IQR Method)'
    if USE_CLAIMS_ONLY:
        title += ' (Claims Data)'
    plt.suptitle(title, fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()
    
    # Print outlier summary
    print("\n" + "="*60)
    print("OUTLIER DETECTION SUMMARY")
    print("="*60)
    for col, stats in outlier_report.items():
        if stats['outliers'] > 0:
            print(f"• {col}: {stats['outliers']:,} outliers ({stats['outlier_pct']}%)")
        else:
            print(f"• {col}: No outliers detected")
    
    # Save outlier report
    with open('reports/outlier_report.json', 'w') as f:
        json.dump(outlier_report, f, indent=2)
    print("💾 Saved outlier report to 'reports/outlier_report.json'")
else:
    print("⚠️ No key columns available for outlier detection")

# Continue with the rest of your notebook...
# [Rest of your notebook code remains the same from section 4 onward]

In [ ]:
# ============================================================================
# DIAGNOSTIC CHECK FOR CLAIMS DATA
# ============================================================================
print("\n" + "="*70)
print("DIAGNOSTIC CHECK FOR CLAIMS DATA")
print("="*70)

# Check TotalClaims values
if 'TotalClaims' in df.columns:
    print("\n🔍 CHECKING TOTALCLAIMS VALUES:")
    print("-" * 40)
    
    # Check data type
    print(f"• Data type: {df['TotalClaims'].dtype}")
    
    # Check basic statistics
    claims_stats = df['TotalClaims'].describe()
    print(f"• Min: {claims_stats['min']:.2f}")
    print(f"• Max: {claims_stats['max']:.2f}")
    print(f"• Mean: {claims_stats['mean']:.2f}")
    print(f"• Std: {claims_stats['std']:.2f}")
    
    # Check if claims > 0
    claims_positive = (df['TotalClaims'] > 0).sum()
    claims_zero = (df['TotalClaims'] == 0).sum()
    claims_negative = (df['TotalClaims'] < 0).sum()
    
    print(f"\n📊 CLAIMS DISTRIBUTION:")
    print(f"• Claims > 0: {claims_positive:,} rows ({claims_positive/len(df)*100:.1f}%)")
    print(f"• Claims = 0: {claims_zero:,} rows ({claims_zero/len(df)*100:.1f}%)")
    print(f"• Claims < 0: {claims_negative:,} rows ({claims_negative/len(df)*100:.1f}%)")
    
    # Show sample of actual claim values
    print(f"\n📋 SAMPLE OF TOTALCLAIMS VALUES:")
    sample_claims = df['TotalClaims'].head(20)
    for i, val in enumerate(sample_claims):
        print(f"  Row {i+1}: {val}")
    
    # Check correlation with TotalPremium
    if 'TotalPremium' in df.columns:
        correlation = df['TotalPremium'].corr(df['TotalClaims'])
        print(f"\n📈 CORRELATION WITH TOTALPREMIUM: {correlation:.4f}")

# Check TotalPremium values
if 'TotalPremium' in df.columns:
    print("\n🔍 CHECKING TOTALPREMIUM VALUES:")
    print("-" * 40)
    
    # Check basic statistics
    premium_stats = df['TotalPremium'].describe()
    print(f"• Min: ${premium_stats['min']:.2f}")
    print(f"• Max: ${premium_stats['max']:.2f}")
    print(f"• Mean: ${premium_stats['mean']:.2f}")
    print(f"• Std: ${premium_stats['std']:.2f}")
    

In [ ]:
# 1.4.2 Enhanced Scatter Plots with Fixes
print("\n📍 ENHANCED SCATTER PLOTS")
print("-" * 40)

# Create scatter plots for important relationships
scatter_pairs = [
    ('TotalPremium', 'TotalClaims'),
    ('SumInsured', 'CalculatedPremiumPerTerm'),
    ('VehicleAge', 'LossRatio')
]

for x_col, y_col in scatter_pairs:
    if x_col in df.columns and y_col in df.columns:
        print(f"\nScatter plot: {x_col} vs {y_col}")
        print("-" * 30)
        
        # Filter out NaN values
        valid_data = df[[x_col, y_col]].dropna()
        
        if len(valid_data) > 10:  # Need enough data points
            print(f"• Data points: {len(valid_data):,}")
            print(f"• {x_col} range: [{valid_data[x_col].min():.2f}, {valid_data[x_col].max():.2f}]")
            print(f"• {y_col} range: [{valid_data[y_col].min():.2f}, {valid_data[y_col].max():.2f}]")
            
            # Check if data needs scaling
            if valid_data[y_col].max() - valid_data[y_col].min() < 0.1:
                print(f"⚠️ Warning: {y_col} has very small range. Checking scaling...")
                
                # Try to identify if values are in thousands/millions
                mean_val = valid_data[y_col].mean()
                if abs(mean_val) < 1:
                    print(f"  • Values appear to be in decimal format")
                    print(f"  • Consider if values need to be multiplied (e.g., by 1000)")
            
            # Calculate correlation
            correlation = valid_data[x_col].corr(valid_data[y_col])
            print(f"• Correlation: {correlation:.4f}")
            
            plt.figure(figsize=(12, 8))
            
            # Create scatter plot with better visualization
            if len(valid_data) > 1000:
                # Use hexbin for large datasets
                plt.hexbin(valid_data[x_col], valid_data[y_col], 
                          gridsize=50, cmap='Blues', mincnt=1)
                plt.colorbar(label='Count')
            else:
                # Regular scatter for smaller datasets
                plt.scatter(valid_data[x_col], valid_data[y_col], 
                          alpha=0.5, s=20, color='steelblue', edgecolor='black')
            
            # Add trend line if we have valid correlation
            if abs(correlation) > 0.1:  # Only add if there's some correlation
                try:
                    z = np.polyfit(valid_data[x_col], valid_data[y_col], 1)
                    p = np.poly1d(z)
                    
                    # Generate x values for trend line
                    x_vals = np.linspace(valid_data[x_col].min(), 
                                        valid_data[x_col].max(), 100)
                    plt.plot(x_vals, p(x_vals), "r--", alpha=0.8, 
                            label=f'Trend: y = {z[0]:.6f}x + {z[1]:.4f}')
                    plt.legend()
                except:
                    print("  ⚠️ Could not calculate trend line")
            
            title = f'{x_col} vs {y_col}'
            if USE_CLAIMS_ONLY:
                title += ' (Claims Data)'
            plt.title(title, fontsize=14)
            plt.xlabel(x_col, fontsize=12)
            plt.ylabel(y_col, fontsize=12)
            plt.grid(True, alpha=0.3)
            
            # Add statistics box
            stats_text = (f'Correlation: {correlation:.4f}\n'
                         f'Data points: {len(valid_data):,}\n'
                         f'Mean {x_col}: {valid_data[x_col].mean():.2f}\n'
                         f'Mean {y_col}: {valid_data[y_col].mean():.2f}')
            
            plt.text(0.05, 0.95, stats_text, transform=plt.gca().transAxes, 
                    fontsize=10, verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
            
            plt.tight_layout()
            plt.show()
            
            # Also create a log-scale version if data spans large ranges
            if (valid_data[x_col].max() / valid_data[x_col].min() > 100 or 
                valid_data[y_col].max() / valid_data[y_col].min() > 100):
                
                print(f"📈 Creating log-scale version for better visualization")
                
                plt.figure(figsize=(12, 8))
                plt.scatter(valid_data[x_col], valid_data[y_col], 
                          alpha=0.5, s=20, color='steelblue', edgecolor='black')
                
                plt.xscale('log')
                plt.yscale('log')
                plt.title(f'{x_col} vs {y_col} (Log Scale)', fontsize=14)
                plt.xlabel(f'{x_col} (log scale)', fontsize=12)
                plt.ylabel(f'{y_col} (log scale)', fontsize=12)
                plt.grid(True, alpha=0.3)
                plt.tight_layout()
                plt.show()
        else:
            print(f"⚠️ Insufficient data for scatter plot: only {len(valid_data)} valid points")
    else:
        print(f"⚠️ Cannot plot {x_col} vs {y_col}: columns not found")

In [ ]:

# rubric_status["1.5 Outlier Detection"] = "Complete"

# ============================================================================
# 4. ADDITIONAL ANALYSES & INSIGHTS
# ============================================================================
print("\n" + "="*70)
print("ADDITIONAL ANALYSES & INSIGHTS")
print("="*70)

# Time Series Analysis
if 'TransactionMonth' in df.columns:
    print("\n📅 TIME SERIES ANALYSIS")
    print("-" * 40)
    
    try:
        # Convert to datetime if needed
        if not pd.api.types.is_datetime64_any_dtype(df['TransactionMonth']):
            df['TransactionMonth'] = pd.to_datetime(df['TransactionMonth'])
        
        # Monthly aggregation
        monthly_data = df.groupby(df['TransactionMonth'].dt.to_period('M')).agg({
            'TotalPremium': 'sum',
            'TotalClaims': 'sum',
            'PolicyID': 'count'
        }).rename(columns={'PolicyID': 'PolicyCount'})
        
        monthly_data['LossRatio'] = monthly_data['TotalClaims'] / monthly_data['TotalPremium']
        
        # Plot time series
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        monthly_data['TotalPremium'].plot(ax=axes[0,0], color='blue', marker='o', linewidth=2)
        axes[0,0].set_title('Monthly Total Premium', fontsize=13)
        axes[0,0].set_ylabel('Premium ($)', fontsize=11)
        axes[0,0].grid(True, alpha=0.3)
        
        monthly_data['TotalClaims'].plot(ax=axes[0,1], color='red', marker='o', linewidth=2)
        axes[0,1].set_title('Monthly Total Claims', fontsize=13)
        axes[0,1].set_ylabel('Claims ($)', fontsize=11)
        axes[0,1].grid(True, alpha=0.3)
        
        monthly_data['PolicyCount'].plot(ax=axes[1,0], color='green', marker='o', linewidth=2)
        axes[1,0].set_title('Monthly Policy Count', fontsize=13)
        axes[1,0].set_ylabel('Number of Policies', fontsize=11)
        axes[1,0].grid(True, alpha=0.3)
        
        monthly_data['LossRatio'].plot(ax=axes[1,1], color='purple', marker='o', linewidth=2)
        axes[1,1].set_title('Monthly Loss Ratio', fontsize=13)
        axes[1,1].set_ylabel('Loss Ratio', fontsize=11)
        axes[1,1].grid(True, alpha=0.3)
        
        title = 'Monthly Insurance Metrics Over Time'
        if USE_CLAIMS_ONLY:
            title += ' (Claims Data)'
        plt.suptitle(title, fontsize=16, y=1.02)
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"⚠️ Could not perform time series analysis: {e}")

# Categorical Analysis
if 'CoverType' in df.columns:
    print("\n🛡️ COVER TYPE ANALYSIS")
    print("-" * 40)
    
    cover_stats = df.groupby('CoverType').agg({
        'TotalPremium': ['count', 'sum', 'mean', 'std'],
        'TotalClaims': ['sum', 'mean'],
        'LossRatio': 'mean'
    }).round(2)
    
    print("Cover Type Statistics (first 10):")
    print(cover_stats.head(10))
    
    # Visualize top cover types
    plt.figure(figsize=(12, 6))
    top_covers = df['CoverType'].value_counts().head(10)
    top_covers.plot(kind='bar', color='lightseagreen')
    title = 'Top 10 Cover Types by Policy Count'
    if USE_CLAIMS_ONLY:
        title += ' (Claims Data)'
    plt.title(title, fontsize=14)
    plt.xlabel('Cover Type', fontsize=12)
    plt.ylabel('Number of Policies', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()


In [ ]:

# ============================================================================
# 5. SAVE RESULTS & GENERATE REPORT
# ============================================================================
print("\n" + "="*70)
print("SAVING RESULTS & GENERATING REPORTS")
print("="*70)

# Create reports directory
os.makedirs('reports', exist_ok=True)

# Save dataset sample
df_sample = df.head(1000)
df_sample.to_csv('reports/data_sample.csv', index=False)
print("Saved data sample to 'reports/data_sample.csv'")

# Generate summary report (using UTF-8 encoding to avoid Unicode errors)
with open('reports/eda_summary.txt', 'w', encoding='utf-8') as f:
    f.write("="*60 + "\n")
    f.write("EDA SUMMARY REPORT\n")
    f.write("="*60 + "\n\n")
    f.write(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Dataset: {df.shape[0]} rows × {df.shape[1]} columns\n")
    f.write(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB\n")
    if USE_CLAIMS_ONLY:
        f.write("Dataset Type: Claims Data Only (TotalClaims > 0)\n")
    else:
        f.write("Dataset Type: Full Dataset (All Policies)\n")
    
    f.write("\nKEY FINDINGS:\n")
    if 'LossRatio' in df.columns:
        f.write(f"- Average Loss Ratio: {df['LossRatio'].mean():.2%}\n")
    if 'TotalClaims' in df.columns:
        if USE_CLAIMS_ONLY:
            f.write(f"- Total Claims (in dataset): ${df['TotalClaims'].sum():,.2f}\n")
            f.write(f"- Average Claim Amount: ${df['TotalClaims'].mean():,.2f}\n")
        else:
            claim_freq = (df['TotalClaims'] > 0).mean()
            f.write(f"- Claim Frequency: {claim_freq:.2%}\n")
            f.write(f"- Total Claims: ${df['TotalClaims'].sum():,.2f}\n")
    if 'TotalPremium' in df.columns:
        f.write(f"- Total Premium: ${df['TotalPremium'].sum():,.2f}\n")
    if 'SumInsured' in df.columns:
        f.write(f"- Total Sum Insured: ${df['SumInsured'].sum():,.2f}\n")

print("Generated summary report at 'reports/eda_summary.txt'")


In [ ]:

# ============================================================================
# 6. FINAL OUTPUT & NEXT STEPS
# ============================================================================
print("\n" + "="*70)
print("EDA COMPLETED SUCCESSFULLY!")
print("="*70)

print("\nWHAT WAS ACCOMPLISHED:")
print("1. [OK] Data loading and preprocessing with DataPreprocessor")
print("2. [OK] Comprehensive descriptive statistics")
print("3. [OK] Data quality assessment with missing value analysis")
print("4. [OK] Univariate analysis with histograms and bar charts")
print("5. [OK] Bivariate analysis with correlation matrix and scatter plots")
print("6. [OK] Outlier detection with box plots")
print("7. [OK] Additional time series and categorical analyses")
print("8. [OK] Reports and visualizations saved")

if USE_CLAIMS_ONLY:
    print("\n⚠️ NOTE: Analysis performed on CLAIMS DATA ONLY (TotalClaims > 0)")
    print("   This is appropriate for analyzing actual claims patterns")
    print("   For overall portfolio analysis, use the full dataset")

print("\nFILES GENERATED:")
print("• reports/descriptive_statistics.csv")
print("• reports/correlation_matrix.csv")
print("• reports/outlier_report.json")
print("• reports/data_sample.csv")
print("• reports/eda_summary.txt")

print("\nNEXT STEPS FOR OTHER RUBRICS:")
print("1. Check repository structure for Rubric 4 requirements")
print("2. Ensure DVC is properly set up (Rubric 2)")
print("3. Document Git practices in commit history (Rubric 3)")
print("4. Update README.md with EDA findings")

print("\n" + "="*70)
print("END OF EDA NOTEBOOK")
print("="*70)


In [ ]:

# ============================================================================
# 7. BUSINESS INSIGHTS ANALYSIS (ANSWERING SPECIFIC QUESTIONS)
# ============================================================================
print("\n" + "="*70)
print("BUSINESS INSIGHTS ANALYSIS")
print("="*70)

# Add note about dataset type
if USE_CLAIMS_ONLY:
    print("⚠️ NOTE: Analyzing CLAIMS DATA ONLY (TotalClaims > 0)")
    print("This is appropriate for claims-specific questions\n")

# ----------------------------------------------------------------------------
# QUESTION 1: Loss Ratio Analysis by Various Dimensions
# ----------------------------------------------------------------------------
print("\n🔍 QUESTION 1: LOSS RATIO ANALYSIS")
print("-" * 40)

# Ensure LossRatio exists or calculate it
if 'LossRatio' not in df.columns and all(col in df.columns for col in ['TotalClaims', 'TotalPremium']):
    print("⚠️ LossRatio column not found. Calculating from TotalClaims/TotalPremium...")
    df['LossRatio'] = df['TotalClaims'] / df['TotalPremium'].replace(0, np.nan)
    df['LossRatio'] = df['LossRatio'].clip(lower=0, upper=10)

if 'LossRatio' in df.columns:
    # Overall Loss Ratio
    overall_loss_ratio = df['LossRatio'].mean()
    if pd.notna(overall_loss_ratio):
        dataset_type = "Claims Data" if USE_CLAIMS_ONLY else "Full Portfolio"
        print(f"📊 Overall {dataset_type} Loss Ratio: {overall_loss_ratio:.2%}")
    else:
        print("⚠️ Cannot calculate overall loss ratio - missing data")
    
    # 1.1 Loss Ratio by Province
    if 'Province' in df.columns:
        print("\n📋 Loss Ratio by Province:")
        print("-" * 30)
        province_loss = df.groupby('Province').agg({
            'TotalPremium': 'sum',
            'TotalClaims': 'sum',
            'PolicyID': 'count'
        }).rename(columns={'PolicyID': 'PolicyCount'})
        
        province_loss['CalculatedLossRatio'] = province_loss['TotalClaims'] / province_loss['TotalPremium'].replace(0, np.nan)
        
        # Filter out provinces with insufficient data
        min_policies = 50  # Minimum number of policies for reliable analysis
        province_loss = province_loss[province_loss['PolicyCount'] >= min_policies]
        
        if len(province_loss) > 0:
            # Sort by Loss Ratio (highest to lowest)
            province_sorted = province_loss.sort_values('CalculatedLossRatio', ascending=False)
            
            print(f"Top 5 Highest Loss Ratio Provinces (min {min_policies} policies):")
            for i, (province, row) in enumerate(province_sorted.head(5).iterrows()):
                if pd.notna(row['CalculatedLossRatio']):
                    print(f"  {i+1}. {province}: {row['CalculatedLossRatio']:.2%} "
                          f"(Premium: ${row['TotalPremium']:,.0f}, Policies: {row['PolicyCount']:,})")
            
            print(f"\nBottom 5 Lowest Loss Ratio Provinces (min {min_policies} policies):")
            for i, (province, row) in enumerate(province_sorted.tail(5).iterrows()):
                if pd.notna(row['CalculatedLossRatio']):
                    print(f"  {i+1}. {province}: {row['CalculatedLossRatio']:.2%} "
                          f"(Premium: ${row['TotalPremium']:,.0f}, Policies: {row['PolicyCount']:,})")
            
            # Visualization
            plt.figure(figsize=(12, 6))
            top_provinces = province_sorted.head(10)
            
            if len(top_provinces) > 0 and 'CalculatedLossRatio' in top_provinces.columns:
                colors = plt.cm.RdYlGn(1 - top_provinces['CalculatedLossRatio'] / top_provinces['CalculatedLossRatio'].max())
                plt.barh(range(len(top_provinces)), top_provinces['CalculatedLossRatio'], color=colors)
                plt.yticks(range(len(top_provinces)), top_provinces.index)
                plt.xlabel('Loss Ratio')
                title = 'Top 10 Provinces by Loss Ratio (Higher = Worse)'
                if USE_CLAIMS_ONLY:
                    title += ' (Claims Data)'
                plt.title(title)
                plt.grid(True, alpha=0.3, axis='x')
                plt.tight_layout()
                plt.show()
        else:
            print(f"⚠️ No provinces with at least {min_policies} policies for reliable analysis")
    
    # 1.2 Loss Ratio by Gender
    if 'Gender' in df.columns:
        print("\n📋 Loss Ratio by Gender:")
        print("-" * 30)
        gender_loss = df.groupby('Gender').agg({
            'TotalPremium': 'sum',
            'TotalClaims': 'sum',
            'PolicyID': 'count'
        }).rename(columns={'PolicyID': 'PolicyCount'})
        
        gender_loss['LossRatio'] = gender_loss['TotalClaims'] / gender_loss['TotalPremium'].replace(0, np.nan)
        
        # Filter out genders with insufficient data
        gender_loss = gender_loss[gender_loss['PolicyCount'] >= 10]
        
        if len(gender_loss) > 0:
            print(gender_loss[['PolicyCount', 'TotalPremium', 'TotalClaims', 'LossRatio']].round(4))
            
            # Visualization
            plt.figure(figsize=(8, 6))
            colors = ['lightblue', 'lightcoral', 'lightgreen']
            gender_loss['LossRatio'].plot(kind='bar', color=colors[:len(gender_loss)])
            title = 'Loss Ratio by Gender'
            if USE_CLAIMS_ONLY:
                title += ' (Claims Data)'
            plt.title(title)
            plt.xlabel('Gender')
            plt.ylabel('Loss Ratio')
            plt.xticks(rotation=0)
            plt.grid(True, alpha=0.3, axis='y')
            plt.tight_layout()
            plt.show()
        else:
            print("⚠️ Insufficient data for gender analysis")
    
    # 1.3 Loss Ratio by VehicleType (if available)
    # Check for vehicle-related columns
    vehicle_cols = [col for col in df.columns if any(keyword in col.lower() for keyword in 
                                                      ['vehicle', 'car', 'auto', 'motor', 'model', 'make'])]
    vehicle_cols = [col for col in vehicle_cols if df[col].nunique() <= 50 and df[col].nunique() > 1]
    
    for vcol in vehicle_cols[:2]:  # Analyze first 2 vehicle-related columns
        print(f"\n📋 Loss Ratio by {vcol}:")
        print("-" * 30)
        vehicle_loss = df.groupby(vcol).agg({
            'TotalPremium': 'sum',
            'TotalClaims': 'sum',
            'PolicyID': 'count'
        }).rename(columns={'PolicyID': 'PolicyCount'})
        
        vehicle_loss['LossRatio'] = vehicle_loss['TotalClaims'] / vehicle_loss['TotalPremium'].replace(0, np.nan)
        
        # Filter out categories with insufficient data
        vehicle_loss = vehicle_loss[vehicle_loss['PolicyCount'] >= 5]
        
        if len(vehicle_loss) > 1:  # Need at least 2 categories for comparison
            # Sort by Loss Ratio
            vehicle_sorted = vehicle_loss.sort_values('LossRatio', ascending=False)
            
            print(f"Top 5 Highest Loss Ratio {vcol} (min 5 policies):")
            for i, (vehicle, row) in enumerate(vehicle_sorted.head(5).iterrows()):
                if pd.notna(row['LossRatio']):
                    print(f"  {i+1}. {vehicle}: {row['LossRatio']:.2%} "
                          f"(Policies: {row['PolicyCount']:,})")


In [ ]:

# ----------------------------------------------------------------------------
# QUESTION 2: Distributions & Outlier Analysis
# ----------------------------------------------------------------------------
print("\n📊 QUESTION 2: FINANCIAL DISTRIBUTIONS & OUTLIERS")
print("-" * 40)

# 2.1 Robust distribution analysis
financial_cols = ['TotalPremium', 'TotalClaims', 'SumInsured', 'CustomValueEstimate']
available_financial = [col for col in financial_cols if col in df.columns]

if available_financial:
    print(f"Analyzing distributions for: {', '.join(available_financial)}")
    
    # Create enhanced distribution plots with error handling
    n_cols = min(2, len(available_financial))
    n_rows = (len(available_financial) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 4*n_rows))
    axes = axes.flatten() if n_rows > 1 or n_cols > 1 else [axes]
    
    for idx, col in enumerate(available_financial):
        if idx < len(axes):
            ax = axes[idx]
            data = df[col].dropna()
            
            if len(data) > 10:  # Need enough data for meaningful analysis
                # Check for variance
                if data.std() > 0:  # Only plot if there's variance
                    # Simple histogram without KDE
                    ax.hist(data, bins=50, alpha=0.7, color='steelblue', edgecolor='black')
                    
                    # Add statistics
                    stats_text = (f"Mean: ${data.mean():,.0f}\n"
                                f"Median: ${data.median():,.0f}\n"
                                f"Std: ${data.std():,.0f}\n"
                                f"Skew: {data.skew():.2f}\n"
                                f"Count: {len(data):,}")
                    if USE_CLAIMS_ONLY and col == 'TotalClaims':
                        stats_text += f"\nClaims Data Only"
                    
                    ax.text(0.95, 0.95, stats_text, transform=ax.transAxes,
                           fontsize=8, verticalalignment='top', horizontalalignment='right',
                           bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
                    
                    ax.set_title(f'Distribution of {col}', fontsize=11)
                    ax.set_xlabel('Value ($)', fontsize=9)
                    ax.set_ylabel('Frequency', fontsize=9)
                    ax.grid(True, alpha=0.3)
                else:
                    ax.text(0.5, 0.5, 'Insufficient variance\nfor distribution plot', 
                           ha='center', va='center', transform=ax.transAxes)
                    ax.set_title(f'{col} (Constant Values)', fontsize=11)
            else:
                ax.text(0.5, 0.5, 'Insufficient data', 
                       ha='center', va='center', transform=ax.transAxes)
                ax.set_title(f'{col} (No Data)', fontsize=11)
    
    # Hide empty subplots
    for idx in range(len(available_financial), len(axes)):
        axes[idx].set_visible(False)
    
    title = 'Financial Variable Distributions'
    if USE_CLAIMS_ONLY:
        title += ' (Claims Data)'
    plt.suptitle(title, fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()

# 2.2 CustomValueEstimate outlier analysis (if available)
if 'CustomValueEstimate' in df.columns:
    print("\n🔍 CustomValueEstimate Outlier Analysis:")
    print("-" * 30)
    
    # Calculate outlier thresholds
    data = df['CustomValueEstimate'].dropna()
    
    if len(data) > 10 and data.std() > 0:  # Need enough data with variance
        Q1 = data.quantile(0.25)
        Q3 = data.quantile(0.75)
        IQR = Q3 - Q1
        
        if IQR > 0:  # Check if IQR is valid
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            outliers = data[(data < lower_bound) | (data > upper_bound)]
            outlier_percentage = (len(outliers) / len(data)) * 100
            
            print(f"• Total values: {len(data):,}")
            print(f"• Outliers (IQR method): {len(outliers):,} ({outlier_percentage:.1f}%)")
            print(f"• Outlier range: <${max(0, lower_bound):,.0f} or >${upper_bound:,.0f}")
            
            if len(outliers) > 0:
                print(f"• Mean of outliers: ${outliers.mean():,.0f}")
                print(f"• Max outlier: ${outliers.max():,.0f}")
            
            # Impact analysis
            if 'TotalClaims' in df.columns and len(outliers) > 0:
                outlier_claims = df.loc[outliers.index, 'TotalClaims'].sum()
                total_claims = df['TotalClaims'].sum()
                if total_claims > 0:
                    print(f"• Outliers account for {outlier_claims/total_claims*100:.1f}% of total claims")
        else:
            print("⚠️ Insufficient variance for outlier detection (IQR = 0)")
    else:
        print("⚠️ Insufficient data or zero variance for analysis")



In [ ]:

# ----------------------------------------------------------------------------
# QUESTION 3: Temporal Trend Analysis (Enhanced)
# ----------------------------------------------------------------------------
print("\n📅 QUESTION 3: ENHANCED TEMPORAL TREND ANALYSIS")
print("-" * 40)

if 'TransactionMonth' in df.columns and 'TotalClaims' in df.columns:
    try:
        # Convert to datetime if needed
        if not pd.api.types.is_datetime64_any_dtype(df['TransactionMonth']):
            df['TransactionMonth'] = pd.to_datetime(df['TransactionMonth'])
        
        # Monthly aggregation with more metrics
        monthly_data = df.groupby(df['TransactionMonth'].dt.to_period('M')).agg({
            'TotalPremium': 'sum',
            'TotalClaims': 'sum',
            'PolicyID': 'count'
        }).rename(columns={'PolicyID': 'PolicyCount'})
        
        monthly_data['CalculatedLossRatio'] = monthly_data['TotalClaims'] / monthly_data['TotalPremium'].replace(0, np.nan)
        
        # Calculate average claim (handle division by zero)
        valid_months = monthly_data['PolicyCount'] > 0
        monthly_data['AverageClaim'] = monthly_data['TotalClaims'] / monthly_data['PolicyCount'].where(valid_months, 1)
        
        # Calculate claim frequency (only for full dataset)
        if not USE_CLAIMS_ONLY:
            claims_by_month = df[df['TotalClaims'] > 0].groupby(df['TransactionMonth'].dt.to_period('M'))['PolicyID'].count()
            monthly_data['ClaimFrequency'] = (claims_by_month / monthly_data['PolicyCount']).fillna(0)
        
        print("📈 Monthly Trend Summary:")
        print("-" * 30)
        print(f"• Time period: {monthly_data.index[0]} to {monthly_data.index[-1]}")
        print(f"• Total months: {len(monthly_data)}")
        if USE_CLAIMS_ONLY:
            print(f"• Dataset: Claims Data Only")
        
        # Calculate trends (only if we have enough data points)
        if len(monthly_data) >= 2:
            x_values = range(len(monthly_data))
            
            # Calculate premium trend
            if monthly_data['TotalPremium'].std() > 0:
                premium_trend = np.polyfit(x_values, monthly_data['TotalPremium'], 1)[0]
                print(f"\n📊 Trend Analysis (slope):")
                print(f"• Premium trend: {'↑' if premium_trend > 0 else '↓'} ${premium_trend:,.0f}/month")
            
            # Calculate claims trend
            if monthly_data['TotalClaims'].std() > 0:
                claims_trend = np.polyfit(x_values, monthly_data['TotalClaims'], 1)[0]
                print(f"• Claims trend: {'↑' if claims_trend > 0 else '↓'} ${claims_trend:,.0f}/month")
            
            # Calculate loss ratio trend
            valid_loss_ratio = monthly_data['CalculatedLossRatio'].dropna()
            if len(valid_loss_ratio) >= 2 and valid_loss_ratio.std() > 0:
                loss_ratio_trend = np.polyfit(range(len(valid_loss_ratio)), valid_loss_ratio, 1)[0]
                print(f"• Loss Ratio trend: {'↑' if loss_ratio_trend > 0 else '↓'} {loss_ratio_trend:.4f}/month")
        
        # Enhanced visualization
        fig, axes = plt.subplots(2, 3, figsize=(16, 10))
        axes = axes.flatten()
        
        metrics = [
            ('TotalPremium', 'blue', 'Monthly Total Premium', 'Premium ($)'),
            ('TotalClaims', 'red', 'Monthly Total Claims', 'Claims ($)'),
            ('PolicyCount', 'green', 'Monthly Policy Count', 'Number of Policies'),
            ('CalculatedLossRatio', 'purple', 'Monthly Loss Ratio', 'Loss Ratio'),
            ('AverageClaim', 'orange', 'Average Claim Amount', 'Average Claim ($)'),
        ]
        
        if not USE_CLAIMS_ONLY:
            metrics.append(('ClaimFrequency', 'brown', 'Claim Frequency', 'Claim Frequency'))
        
        plot_count = 0
        for idx, (metric, color, title, ylabel) in enumerate(metrics):
            if idx < len(axes) and metric in monthly_data.columns:
                ax = axes[plot_count]
                if len(monthly_data[metric].dropna()) > 0:
                    monthly_data[metric].plot(ax=ax, color=color, marker='o', linewidth=2)
                    ax.set_title(title, fontsize=12)
                    ax.set_ylabel(ylabel, fontsize=10)
                    ax.grid(True, alpha=0.3)
                    plot_count += 1
                else:
                    ax.text(0.5, 0.5, 'No data available', 
                           ha='center', va='center', transform=ax.transAxes)
                    ax.set_title(title, fontsize=12)
                    plot_count += 1
        
        # Hide any empty axes
        for idx in range(plot_count, len(axes)):
            axes[idx].set_visible(False)
        
        title = 'Comprehensive Monthly Insurance Trends'
        if USE_CLAIMS_ONLY:
            title += ' (Claims Data)'
        plt.suptitle(title, fontsize=16, y=1.02)
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"⚠️ Could not perform enhanced temporal analysis: {e}")
        import traceback
        traceback.print_exc()


# ----------------------------------------------------------------------------
# QUESTION 4: Vehicle Make/Model Analysis
# ----------------------------------------------------------------------------
print("\n🚗 QUESTION 4: VEHICLE MAKE/MODEL ANALYSIS")
print("-" * 40)

# Identify potential vehicle-related columns
vehicle_make_cols = [col for col in df.columns if any(keyword in col.lower() for keyword in ['make', 'model', 'manufacturer', 'brand'])]
vehicle_type_cols = [col for col in df.columns if any(keyword in col.lower() for keyword in ['type', 'category', 'class'])]

print(f"Potential vehicle make/model columns: {vehicle_make_cols}")
print(f"Potential vehicle type columns: {vehicle_type_cols}")

# Analyze available vehicle columns
vehicle_analysis_results = {}

for vcol in vehicle_make_cols + vehicle_type_cols:
    if vcol in df.columns:
        unique_count = df[vcol].nunique()
        # Only analyze if we have reasonable number of categories and data
        if 2 <= unique_count <= 50 and df[vcol].count() > 100:  
            print(f"\n📊 Analyzing {vcol} ({unique_count} unique values, {df[vcol].count():,} records):")
            print("-" * 40)
            
            try:
                vehicle_stats = df.groupby(vcol).agg({
                    'TotalPremium': ['count', 'sum', 'mean'],
                    'TotalClaims': ['sum', 'mean'],
                })
                
                # Flatten column names
                vehicle_stats.columns = [f'{col[0]}_{col[1]}' for col in vehicle_stats.columns]
                
                # Calculate Loss Ratio (handle division by zero)
                vehicle_stats['LossRatio'] = vehicle_stats['TotalClaims_sum'] / vehicle_stats['TotalPremium_sum'].replace(0, np.nan)
                
                # Filter out categories with insufficient data
                vehicle_stats = vehicle_stats[vehicle_stats['TotalPremium_count'] >= 10]  # At least 10 policies
                
                if len(vehicle_stats) > 1:  # Need at least 2 categories for comparison
                    # Sort by different metrics
                    by_loss_ratio = vehicle_stats.sort_values('LossRatio', ascending=False)
                    by_claims = vehicle_stats.sort_values('TotalClaims_sum', ascending=False)
                    by_premium = vehicle_stats.sort_values('TotalPremium_sum', ascending=False)
                    
                    print(f"Top 5 by Loss Ratio (Highest Risk):")
                    for i, (idx, row) in enumerate(by_loss_ratio.head(5).iterrows()):
                        if pd.notna(row['LossRatio']):
                            print(f"  {i+1}. {idx}: {row['LossRatio']:.2%} "
                                  f"(Claims: ${row['TotalClaims_sum']:,.0f}, Policies: {row['TotalPremium_count']:,})")
                    
                    print(f"\nTop 5 by Total Claims (Highest Cost):")
                    for i, (idx, row) in enumerate(by_claims.head(5).iterrows()):
                        print(f"  {i+1}. {idx}: ${row['TotalClaims_sum']:,.0f} "
                              f"(Policies: {row['TotalPremium_count']:,})")
                    
                    print(f"\nTop 5 by Total Premium (Highest Revenue):")
                    for i, (idx, row) in enumerate(by_premium.head(5).iterrows()):
                        print(f"  {i+1}. {idx}: ${row['TotalPremium_sum']:,.0f} "
                              f"(Policies: {row['TotalPremium_count']:,})")
                    
                    # Store for visualization
                    vehicle_analysis_results[vcol] = vehicle_stats
                    
                    # Visualization (only if we have data)
                    if len(by_loss_ratio) > 0 and len(by_claims) > 0:
                        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
                        
                        # Top 10 by Loss Ratio
                        top_loss = by_loss_ratio.head(10)
                        if len(top_loss) > 0:
                            colors = plt.cm.RdYlGn(1 - top_loss['LossRatio'] / top_loss['LossRatio'].max())
                            axes[0].barh(range(len(top_loss)), top_loss['LossRatio'], color=colors)
                            axes[0].set_yticks(range(len(top_loss)))
                            axes[0].set_yticklabels(top_loss.index)
                            axes[0].set_xlabel('Loss Ratio')
                            title = f'Top 10 {vcol} by Loss Ratio'
                            if USE_CLAIMS_ONLY:
                                title += ' (Claims)'
                            axes[0].set_title(title)
                            axes[0].grid(True, alpha=0.3, axis='x')
                        
                        # Top 10 by Total Claims
                        top_claims = by_claims.head(10)
                        if len(top_claims) > 0:
                            axes[1].bar(range(len(top_claims)), top_claims['TotalClaims_sum'] / 1000, color='lightcoral')
                            axes[1].set_xticks(range(len(top_claims)))
                            axes[1].set_xticklabels(top_claims.index, rotation=45, ha='right')
                            axes[1].set_ylabel('Total Claims (Thousands $)')
                            title = f'Top 10 {vcol} by Total Claims'
                            if USE_CLAIMS_ONLY:
                                title += ' (Claims)'
                            axes[1].set_title(title)
                            axes[1].grid(True, alpha=0.3, axis='y')
                        
                        plt.suptitle(f'Vehicle Analysis: {vcol}', fontsize=14, y=1.02)
                        plt.tight_layout()
                        plt.show()
            except Exception as e:
                print(f"⚠️ Error analyzing {vcol}: {e}")
                continue

# If no specific vehicle columns found, check for general vehicle info
if not vehicle_analysis_results and 'VehicleAge' in df.columns:
    print("\n📊 Vehicle Age Analysis (since specific make/model columns not found):")
    print("-" * 40)
    
    try:
        # Create age bins
        df['VehicleAgeGroup'] = pd.cut(df['VehicleAge'], 
                                       bins=[0, 3, 5, 7, 10, 15, 30, 100], 
                                       labels=['0-3', '4-5', '6-7', '8-10', '11-15', '16-30', '30+'])
        
        age_stats = df.groupby('VehicleAgeGroup').agg({
            'TotalPremium': ['count', 'sum', 'mean'],
            'TotalClaims': ['sum', 'mean'],
            'LossRatio': 'mean'
        }).round(4)
        
        print("Vehicle Age Group Analysis:")
        print(age_stats)
    except Exception as e:
        print(f"⚠️ Error in vehicle age analysis: {e}")



In [ ]:

# ============================================================================
# 8. BUSINESS RECOMMENDATIONS
# ============================================================================
print("\n" + "="*70)
print("BUSINESS RECOMMENDATIONS")
print("="*70)

print("\n🎯 KEY RECOMMENDATIONS BASED ON ANALYSIS:")
print("-" * 40)

if USE_CLAIMS_ONLY:
    print("📊 Analysis based on CLAIMS DATA ONLY (policies with TotalClaims > 0):")
    print("  • Insights reflect actual claim patterns, not overall portfolio")
    print("  • Use for claims management and risk assessment")
    print("  • For pricing decisions, combine with full portfolio analysis\n")

# Recommendation 1: Based on Loss Ratio analysis
if 'Province' in df.columns and 'LossRatio' in df.columns:
    try:
        province_loss = df.groupby('Province')['LossRatio'].mean()
        if len(province_loss) > 0:
            high_loss_provinces = province_loss.nlargest(3)
            print("1. 🔴 High Risk Areas (Based on Loss Ratio):")
            for province, loss_ratio in high_loss_provinces.items():
                if pd.notna(loss_ratio) and loss_ratio > 0:
                    if USE_CLAIMS_ONLY:
                        print(f"   • {province}: High claims loss ratio ({loss_ratio:.2%}) - review claims patterns")
                    else:
                        print(f"   • {province}: Consider premium adjustments (Current Loss Ratio: {loss_ratio:.2%})")
    except:
        pass

# Recommendation 2: Based on temporal trends
if 'TransactionMonth' in df.columns and 'TotalClaims' in df.columns:
    print("\n2. 📈 Temporal Recommendations:")
    try:
        monthly_data = df.groupby(df['TransactionMonth'].dt.to_period('M'))['TotalClaims'].sum()
        if len(monthly_data) >= 6:
            recent_avg = monthly_data[-6:].mean()
            previous_avg = monthly_data[-12:-6].mean() if len(monthly_data) >= 12 else monthly_data[:-6].mean()
            trend = "increasing" if recent_avg > previous_avg else "decreasing"
            if USE_CLAIMS_ONLY:
                print(f"   • Claim amounts are {trend} recently. Monitor for seasonal patterns in claims.")
            else:
                print(f"   • Claim amounts are {trend} recently. Monitor for seasonal patterns.")
    except:
        pass

# Recommendation 3: Based on vehicle analysis
if vehicle_analysis_results:
    print("\n3. 🚗 Vehicle Portfolio Recommendations:")
    for vcol, stats in vehicle_analysis_results.items():
        if 'LossRatio' in stats.columns:
            high_risk = stats.nlargest(3, 'LossRatio')
            for idx, row in high_risk.iterrows():
                if pd.notna(row['LossRatio']) and row['LossRatio'] > 0.5:  # 50% loss ratio threshold
                    if USE_CLAIMS_ONLY:
                        print(f"   • {vcol}: {idx} has high claims loss ratio ({row['LossRatio']:.2%})")
                    else:
                        print(f"   • {vcol}: {idx} has high loss ratio ({row['LossRatio']:.2%})")

# Recommendation 4: Based on dataset type
if USE_CLAIMS_ONLY:
    print("\n4. 📊 Claims Data Specific Recommendations:")
    print("   • Use this analysis for claims reserving and claims handling optimization")
    print("   • Identify high-frequency claim types for preventive measures")
    print("   • Analyze claim amounts to set appropriate excess levels")
else:
    print("\n4. ⚠️ Risk Management Recommendations:")
    print("   • Review policies with extreme CustomValueEstimate outliers")
    print("   • Consider implementing value caps for high-risk assets")



In [ ]:

# ============================================================================
# 9. UPDATE SUMMARY REPORT WITH BUSINESS INSIGHTS
# ============================================================================
print("\n" + "="*70)
print("UPDATING SUMMARY REPORT WITH BUSINESS INSIGHTS")
print("="*70)

# Update the summary report
try:
    with open('reports/eda_summary.txt', 'a', encoding='utf-8') as f:
        f.write("\n" + "="*60 + "\n")
        f.write("BUSINESS INSIGHTS & ANSWERS TO QUESTIONS\n")
        f.write("="*60 + "\n\n")
        
        f.write("QUESTION 1: LOSS RATIO ANALYSIS\n")
        f.write("-" * 40 + "\n")
        if 'LossRatio' in df.columns:
            overall_lr = df['LossRatio'].mean()
            if pd.notna(overall_lr):
                if USE_CLAIMS_ONLY:
                    f.write(f"• Loss Ratio for Claims Data: {overall_lr:.2%}\n")
                else:
                    f.write(f"• Overall Portfolio Loss Ratio: {overall_lr:.2%}\n")
            if 'Province' in df.columns:
                province_stats = df.groupby('Province')['LossRatio'].mean()
                if len(province_stats) > 0:
                    f.write(f"• Highest Loss Ratio Province: {province_stats.idxmax()} ({province_stats.max():.2%})\n")
                    f.write(f"• Lowest Loss Ratio Province: {province_stats.idxmin()} ({province_stats.min():.2%})\n")
        
        f.write("\nQUESTION 2: FINANCIAL DISTRIBUTIONS & OUTLIERS\n")
        f.write("-" * 40 + "\n")
        if 'CustomValueEstimate' in df.columns:
            data = df['CustomValueEstimate'].dropna()
            if len(data) > 10 and data.std() > 0:
                Q1 = data.quantile(0.25)
                Q3 = data.quantile(0.75)
                IQR = Q3 - Q1
                if IQR > 0:
                    outliers = data[(data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR))]
                    f.write(f"• CustomValueEstimate outliers: {len(outliers):,} ({len(outliers)/len(data)*100:.1f}%)\n")
        
        f.write("\nQUESTION 3: TEMPORAL TRENDS\n")
        f.write("-" * 40 + "\n")
        if 'TransactionMonth' in df.columns:
            f.write(f"• Data covers: {df['TransactionMonth'].min()} to {df['TransactionMonth'].max()}\n")
            if 'TotalClaims' in df.columns:
                monthly_claims = df.groupby(df['TransactionMonth'].dt.to_period('M'))['TotalClaims'].sum()
                if len(monthly_claims) > 1:
                    f.write(f"• Claim trend: {('Increasing' if monthly_claims.iloc[-1] > monthly_claims.iloc[0] else 'Decreasing')}\n")
        
        f.write("\nQUESTION 4: VEHICLE ANALYSIS\n")
        f.write("-" * 40 + "\n")
        if vehicle_analysis_results:
            for vcol, stats in vehicle_analysis_results.items():
                if 'LossRatio' in stats.columns:
                    high_risk = stats.nlargest(1, 'LossRatio')
                    for idx, row in high_risk.iterrows():
                        if pd.notna(row['LossRatio']):
                            f.write(f"• Highest risk {vcol}: {idx} (Loss Ratio: {row['LossRatio']:.2%})\n")
    
    print("✅ Business insights added to summary report")
except Exception as e:
    print(f"⚠️ Error updating summary report: {e}")



In [ ]:

# ============================================================================
# 10. FINAL COMPLETION CHECK
# ============================================================================
print("\n" + "="*70)
print("FINAL COMPLETION CHECK")
print("="*70)

print("\n✅ ALL QUESTIONS ANSWERED:")
print("1. ✅ Loss Ratio analysis by Province, Gender, and Vehicle types")
print("2. ✅ Detailed financial distributions and outlier analysis")
print("3. ✅ Enhanced temporal trend analysis with claim frequency/severity")
print("4. ✅ Vehicle make/model risk assessment")

if USE_CLAIMS_ONLY:
    print("\n📊 SPECIAL NOTE:")
    print("• Analysis performed on CLAIMS DATA ONLY (TotalClaims > 0)")
    print("• This provides insights into actual claims patterns")
    print("• For overall portfolio analysis, run with full dataset")

print("\n📁 ADDITIONAL FILES GENERATED:")
print("• Enhanced visualizations for business questions")
print("• Updated summary report with business insights")
print("• Risk assessment recommendations")

print("\n" + "="*70)
print("ANALYSIS COMPLETE - ALL BUSINESS QUESTIONS ANSWERED!")
print("="*70)